In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv("traffic.csv")
df.head()


,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


In [4]:
df['DateTime'] = pd.to_datetime(df['DateTime'])


In [5]:
df['hour'] = df['DateTime'].dt.hour
df['day_of_week'] = df['DateTime'].dt.dayofweek  # Monday=0
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
#extarct time-based features

In [6]:
def peak_hour(hour):
    if (8 <= hour <= 10) or (17 <= hour <= 20):
        return 1
    return 0

df['is_peak_hour'] = df['hour'].apply(peak_hour)


In [7]:
le = LabelEncoder()
df['junction_encoded'] = le.fit_transform(df['Junction'])


In [8]:
df.rename(columns={'Vehicles': 'vehicle_count'}, inplace=True)


In [9]:
df['vehicle_count'].describe()


count    48120.000000
mean        22.791334
std         20.750063
min          1.000000
25%          9.000000
50%         15.000000
75%         29.000000
max        180.000000
Name: vehicle_count, dtype: float64

In [15]:
if 'ID' in df.columns:
    df.drop(columns=['ID'], inplace=True)


In [20]:
df['vehicle_count'] = df['vehicle_count'].fillna(df['vehicle_count'].median())
df = df.dropna()


In [18]:
df.columns

Index(['DateTime', 'Junction', 'vehicle_count', 'hour', 'day_of_week',
       'is_weekend', 'is_peak_hour', 'junction_encoded'],
      dtype='object')

In [21]:
def congestion_level(count):
    if count < 30:
        return 0   # Low
    elif count <= 60:
        return 1   # Medium
    else:
        return 2   # High

df['congestion_level'] = df['vehicle_count'].apply(congestion_level)


In [22]:
X = df[['junction_encoded', 'hour', 'is_weekend', 'is_peak_hour', 'vehicle_count']]
y = df['congestion_level']


In [23]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [25]:
print("Training data:", X_train.shape, y_train.shape)
print("Testing data:", X_test.shape, y_test.shape)


Training data: (38496, 5) (38496,)
Testing data: (9624, 5) (9624,)


In [28]:
!python -m pip install requests


In [29]:
import requests


In [32]:
API_KEY = "55211b9ad0ea715f4aebdd43769e42a9"

def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url).json()
    
    weather_main = response['weather'][0]['main']
    temperature = response['main']['temp']
    
    is_rain = 1 if weather_main == 'Rain' else 0
    is_fog = 1 if weather_main in ['Fog', 'Mist', 'Haze'] else 0
    
    return is_rain, is_fog, temperature


In [33]:
city = "Hyderabad"   # or any city you want
is_rain, is_fog, temperature = get_weather(city)

is_rain, is_fog, temperature


(0, 1, 23.23)

In [34]:
df['is_rain'] = is_rain
df['is_fog'] = is_fog
df['temperature'] = temperature


In [35]:
df[['is_rain', 'is_fog', 'temperature']].head()


,is_rain,is_fog,temperature
0,0,1,23.23
1,0,1,23.23
2,0,1,23.23
3,0,1,23.23
4,0,1,23.23


In [36]:
X = df[
    [
        'junction_encoded',
        'hour',
        'is_weekend',
        'is_peak_hour',
        'vehicle_count',
        'is_rain',
        'is_fog',
        'temperature'
    ]
]

y = df['congestion_level']


In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle


In [39]:
X = df[
    [
        'junction_encoded',
        'hour',
        'is_weekend',
        'is_peak_hour',
        'vehicle_count',
        'is_rain',
        'is_fog',
        'temperature'
    ]
]

y = df['congestion_level']


In [40]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle


In [44]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)


In [45]:
rf_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, random_state=42)

In [46]:
y_pred = rf_model.predict(X_test)


In [47]:
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


Model Accuracy: 1.0


In [48]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7263
           1       1.00      1.00      1.00      1720
           2       1.00      1.00      1.00       641

    accuracy                           1.00      9624
   macro avg       1.00      1.00      1.00      9624
weighted avg       1.00      1.00      1.00      9624



In [49]:
confusion_matrix(y_test, y_pred)


array([[7263,    0,    0],
       [   0, 1720,    0],
       [   0,    0,  641]])

In [50]:
sample_input = [[
    2,   # junction_encoded
    18,  # hour
    0,   # is_weekend
    1,   # is_peak_hour
    75,  # vehicle_count
    1,   # is_rain
    0,   # is_fog
    28   # temperature
]]


In [51]:
sample_scaled = scaler.transform(sample_input)


C:\Users\preet\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [52]:
y_pred = rf_model.predict(X_test)


In [53]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
accuracy


1.0

In [54]:
accuracy_percentage = accuracy * 100
print(f"Model Accuracy: {accuracy_percentage:.2f}%")


Model Accuracy: 100.00%


In [55]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7263
           1       1.00      1.00      1.00      1720
           2       1.00      1.00      1.00       641

    accuracy                           1.00      9624
   macro avg       1.00      1.00      1.00      9624
weighted avg       1.00      1.00      1.00      9624



In [56]:
confusion_matrix(y_test, y_pred)


array([[7263,    0,    0],
       [   0, 1720,    0],
       [   0,    0,  641]])

In [57]:
with open("traffic_congestion_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)
#dumping into pickle

In [58]:
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [59]:
with open("traffic_congestion_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    loaded_scaler = pickle.load(f)

sample = X_test[0].reshape(1, -1)
loaded_model.predict(sample)


array([0])

In [63]:
def predict_congestion(
    junction_encoded,
    hour,
    is_weekend,
    is_peak_hour,
    vehicle_count,
    is_rain,
    is_fog,
    temperature
):
    input_df = pd.DataFrame([{
        'junction_encoded': junction_encoded,
        'hour': hour,
        'is_weekend': is_weekend,
        'is_peak_hour': is_peak_hour,
        'vehicle_count': vehicle_count,
        'is_rain': is_rain,
        'is_fog': is_fog,
        'temperature': temperature
    }])
    
    input_scaled = scaler.transform(input_df)
    prediction = rf_model.predict(input_scaled)[0]
    
    return prediction


In [64]:
def congestion_label(pred):
    if pred == 0:
        return "Low Congestion 🟢"
    elif pred == 1:
        return "Medium Congestion 🟡"
    else:
        return "High Congestion 🔴"


In [65]:
pred = predict_congestion(
    junction_encoded=1,
    hour=18,
    is_weekend=0,
    is_peak_hour=1,
    vehicle_count=80,
    is_rain=1,
    is_fog=0,
    temperature=27
)

congestion_label(pred)


'High Congestion 🔴'

In [66]:
test_cases = [
    (0, 9, 0, 1, 25, 0, 0, 30),   # Morning, low traffic
    (2, 18, 0, 1, 70, 1, 0, 28),  # Evening, rain
    (1, 14, 1, 0, 40, 0, 0, 32)   # Afternoon, weekend
]

for case in test_cases:
    pred = predict_congestion(*case)
    print(congestion_label(pred))


Low Congestion 🟢
High Congestion 🔴
Medium Congestion 🟡


In [67]:
def generate_alert(prediction):
    if prediction == 0:
        return {
            "level": "Low Congestion",
            "color": "Green",
            "icon": "🟢",
            "message": "Traffic is smooth. You can proceed normally."
        }
    elif prediction == 1:
        return {
            "level": "Medium Congestion",
            "color": "Yellow",
            "icon": "🟡",
            "message": "Moderate traffic detected. Consider planning ahead."
        }
    else:
        return {
            "level": "High Congestion",
            "color": "Red",
            "icon": "🔴",
            "message": "Heavy traffic ahead! Consider alternative routes."
        }


In [68]:
pred = predict_congestion(
    junction_encoded=1,
    hour=18,
    is_weekend=0,
    is_peak_hour=1,
    vehicle_count=80,
    is_rain=1,
    is_fog=0,
    temperature=27
)

alert = generate_alert(pred)
alert


{'level': 'High Congestion',
 'color': 'Red',
 'icon': '🔴',
 'message': 'Heavy traffic ahead! Consider alternative routes.'}

In [69]:
print(alert["icon"], alert["level"])
print(alert["message"])


🔴 High Congestion
Heavy traffic ahead! Consider alternative routes.


In [70]:
scenarios = [
    (0, 9, 0, 1, 25, 0, 0, 30),   # Low traffic
    (2, 14, 0, 0, 45, 0, 0, 33),  # Medium traffic
    (1, 18, 0, 1, 85, 1, 0, 28)   # High traffic
]

for s in scenarios:
    pred = predict_congestion(*s)
    alert = generate_alert(pred)
    print(alert["icon"], alert["level"], "-", alert["message"])


🟢 Low Congestion - Traffic is smooth. You can proceed normally.
🟡 Medium Congestion - Moderate traffic detected. Consider planning ahead.
🔴 High Congestion - Heavy traffic ahead! Consider alternative routes.


In [71]:
junction_graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2, 4],
    4: [3]
}


In [72]:
junction_congestion = {
    0: "Low",
    1: "High",
    2: "Medium",
    3: "Low",
    4: "Low"
}


In [73]:
def suggest_alternative_route(current_junction):
    alternatives = []
    
    for neighbor in junction_graph.get(current_junction, []):
        congestion = junction_congestion.get(neighbor, "High")
        
        if congestion != "High":
            alternatives.append(neighbor)
    
    return alternatives


In [74]:
current_junction = 1  # example

if junction_congestion[current_junction] == "High":
    routes = suggest_alternative_route(current_junction)
    routes


In [75]:
if routes:
    print("🚦 High congestion detected.")
    print("Suggested alternative junctions:", routes)
else:
    print("No alternative routes available.")


🚦 High congestion detected.
Suggested alternative junctions: [0, 3]


In [76]:
pred = predict_congestion(
    junction_encoded=1,
    hour=18,
    is_weekend=0,
    is_peak_hour=1,
    vehicle_count=85,
    is_rain=1,
    is_fog=0,
    temperature=27
)

alert = generate_alert(pred)
print(alert["icon"], alert["level"])
print(alert["message"])

if alert["level"] == "High Congestion":
    alt_routes = suggest_alternative_route(1)
    print("🛣 Alternative routes suggested:", alt_routes)


🔴 High Congestion
Heavy traffic ahead! Consider alternative routes.
🛣 Alternative routes suggested: [0, 3]
